# Bootstrapping user preferences



Similarly to (Zhu et al., 2021), we'll evaluate a dynamic recommendation system using:

1. An oracle preference model (Generated in notebook 00-preference_model)
2. A position bias examination model
3. Bootstrapped user preferences

Step 3 is important so that we're simulating exposing users to fresh items, using their previous preferences (provided by step 1) and their examination bias (step 2) to determine a sinthethical rating matrix dataset that we'll use to compare different calibration approaches as the user gets exposed to more and more items.

We'll also simulate the timestamp of user interaction, with the following methodology:

1. We'll analyze the average delta in timestamps between consecutive ratings per user in the same dataset that the preference model was trained on (e.g: movielens) (present in data/movielens-1m/avg_str_time_dff_per_user.csv)
2. Assuming that the time_delta is normally distributed (with the mean and standart deviation being defined from the dataset), we'll randomly generate a delta_t for each user. the time_0 will be the timestamp when we start generating the user preferences.

In [1]:
import pandas as pd
from scipy.stats import expon
from tqdm import tqdm

In [2]:
import numpy as np
import math
import random

In [3]:
import sys
import os
from pathlib import Path
import pickle

sys.path.append('..')

In [4]:
from simulationConstants import ML_1M_FILLED_PATH

## Reading and setting up data

In [5]:
preference_matrix = pd.read_csv(f"../{ML_1M_FILLED_PATH}")
preference_matrix = preference_matrix.rename(columns={"Rating": "rating"})


unique_users = list(preference_matrix["user"].unique())

unique_items = list(preference_matrix["item"].unique())


click_matrix = pd.DataFrame(columns=["user", "item", "clicked_and_examined", "clicked_at", "timestamp"])

user_to_time_delta = pd.read_csv("../data/movielens-1m/median_time_diff_per_user.csv").set_index("userId")

In [6]:


userToExpDistribution = {
    user: expon(scale=row["median_timestamp_diff"])
    for user, row in user_to_time_delta.iterrows()
}




with open("userToExpDistribution.pkl", "wb") as f:
    pickle.dump(userToExpDistribution, f)

## Functions

In [7]:
from simulationUtils import map_recommendation_to_feedback

In [8]:
def random_rec(items, u, k, D):
    user_history = set(D[D["user"] == u]["item"])
    candidate_items = list(set(items) - user_history)
    return random.sample(candidate_items, k)

In [9]:
initial_date = None

In [10]:
if initial_date is None:
    initial_date = pd.Timestamp.now().timestamp()
user_to_up_to_date_timestamp = pd.DataFrame({
    "user": unique_users, 
    "delta_from_start": 0.0
})
user_to_up_to_date_timestamp["timestamp_dist"] = user_to_up_to_date_timestamp["user"].map(userToExpDistribution)

In [11]:
pd.Timestamp.now().timestamp()

1762101022.543013

In [12]:
user_to_up_to_date_timestamp

,user,delta_from_start,timestamp_dist
0,1,0.0,<scipy.stats._distn_infrastructure.rv_continuo...
1,2,0.0,<scipy.stats._distn_infrastructure.rv_continuo...
2,3,0.0,<scipy.stats._distn_infrastructure.rv_continuo...
3,5,0.0,<scipy.stats._distn_infrastructure.rv_continuo...
4,6,0.0,<scipy.stats._distn_infrastructure.rv_continuo...
...,...,...,...
5284,6035,0.0,<scipy.stats._distn_infrastructure.rv_continuo...
5285,6036,0.0,<scipy.stats._distn_infrastructure.rv_continuo...
5286,6037,0.0,<scipy.stats._distn_infrastructure.rv_continuo...
5287,6039,0.0,<scipy.stats._distn_infrastructure.rv_continuo...


In [13]:
def simulate_user_feedback(user, candidate_items, click_df, preference_matrix, k, user_to_up_to_date_timestamp, userToExpDistribution):
    rec = random_rec(candidate_items, user, k, click_df)
    # Generates a user feedback to each recommendation and the last recorded time of interaction in this recommendation
    row, last_time = map_recommendation_to_feedback(user, rec, preference_matrix, user_to_up_to_date_timestamp, userToExpDistribution)
    user_to_up_to_date_timestamp.loc[user_to_up_to_date_timestamp["user"] == user, "delta_from_start"] = last_time
    return row, user_to_up_to_date_timestamp

In [14]:
def bootstrap_clicks(D, unique_users, unique_items, preference_matrix, userToExpDistribution, k=20, rounds=10, initial_date=None):
    """
    Given unique users and unique items, recommend up to k items to every user
    using a preference matrix as a relevancy model and using a click model
    to simulate probability of user examinating an item.

    Feedback signal will be fed to the D matrix.

    We run the boostrap process for a total of an arbitrary number of rounds,
    in order to ensure enough feedback data to train a model.
    """


    if initial_date is None:
        initial_date = pd.Timestamp.now().timestamp()

    # Setup a dataframe which maps each user to the time delta from its first interaction.
    user_to_up_to_date_timestamp = pd.DataFrame({
        "user": unique_users,
        "delta_from_start": 0.0
    })
    
    # Maps each user to its corresponding exponential distribution, which models the average time between interactions
    # for each user
    user_to_up_to_date_timestamp["timestamp_dist"] = user_to_up_to_date_timestamp["user"].map(userToExpDistribution)
    new_df = D.copy()
    for round in range(rounds):
        rows_to_append = []
        for user in tqdm(unique_users, desc=f"Processing users (round {round+1}/{rounds})..."):
            row, user_to_up_to_date_timestamp = simulate_user_feedback(user, unique_items, click_matrix, preference_matrix, k, user_to_up_to_date_timestamp, userToExpDistribution)
            #row, user_to_up_to_date_timestamp = map_recommendation_to_feedback(user, recs, preference_matrix, user_to_up_to_date_timestamp)
            rows_to_append.extend(row)
        round_df = pd.DataFrame(rows_to_append, columns=new_df.columns)
        new_df = pd.concat([new_df, round_df], ignore_index=True)
    final_df = pd.concat([D, new_df])
    # Offsets the recorded timestamps by initial_date
    final_df.loc[final_df["timestamp"].notnull(), "timestamp"] += initial_date
    return final_df

In [ ]:
%%time
bootstrapped_clicks = bootstrap_clicks(click_matrix, unique_users, unique_items, preference_matrix, userToExpDistribution=userToExpDistribution, initial_date=0.0, rounds=10)

Processing users (round 1/10)...:   2%|▏         | 83/5289 [00:13<14:57,  5.80it/s]

In [ ]:
bootstrapped_clicks

,user,item,clicked_and_examined,clicked_at,timestamp


In [ ]:
bootstrapped_clicks.to_csv("../data/simulation/movielens_1m_sinthetically_filled.csv", index=False)